### 必要库

In [40]:
%matplotlib inline
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as T
from torchvision import models
from torchvision.io import read_image
import matplotlib.pyplot as plt
import matplotlib
import time


print(torch.__version__)

1.13.0


### 引入resnet预训练模型

In [41]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [42]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

model.to(device)
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)

# model

d:\anaconda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\anaconda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [43]:
label_map = {'CNV':0, 'DME':1, 'DRUSEN':2, 'NORMAL':3,}

def dict_reverse(dict, value):
   return [k for k,v in dict.items() if value==v]
   
pred = dict_reverse(label_map,1)
print(pred[0])

DME


In [55]:
model.load_state_dict(torch.load('OCT-Resnet_model.pth'))
# label_map = {'CNV':0, 'DME':1, 'DRUSEN':2, 'NORMAL':3,}
label_reverse = { 0:'CNV', 1:'DME', 2:'DRUSEN', 3:'NORAML',}

# CNV:脉络膜新生血管
# DME:糖尿病性黄斑水肿
# DRUSEN:玻璃疣
# NORMAL:正常

img_path = r'C:\Users\LRRRR\Desktop\resnet-OCT\templates\DRUSEN-1112835-1.jpeg'
plt.figure()
plt.imread(img_path)
plt.show()

def transform_img(img_path):
        image = read_image(img_path)
        print(type(image))
        # image = image.squeeze()
        print(image.size()[0])
        if image.size()[0] == 1:
                image = image.repeat(3, 1, 1)
        print(image.size())
        image = image / 255.0
        #print(image)
        image = T.Resize((224, 224))(image)
        image = image.to(device)
        image = image.unsqueeze(0)
        print(image.size())
        return image

if __name__=="__main__":
        transform_img(img_path)


<Figure size 640x480 with 0 Axes>

<class 'torch.Tensor'>
3
torch.Size([3, 343, 512])
torch.Size([1, 3, 224, 224])


In [53]:
img_path = r'C:\Users\LRRRR\Desktop\resnet-OCT\templates\CNV-1016042-1.jpeg'
with torch.set_grad_enabled(False):
        outputs = model(transform_img(img_path))
        _, preds = torch.max(outputs, 1)
        prediction = preds.tolist()[0]
        print(prediction)
        print(label_reverse[prediction])

<class 'torch.Tensor'>
3
torch.Size([3, 289, 512])
torch.Size([1, 3, 224, 224])
0
CNV
